In [109]:
import random
import math
import numpy as np
from itertools import chain, combinations
import itertools
def powerset(iterable):
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

In [132]:
class culo_game:
    
    def __init__(self,n_j):
        
        self.players_data = []
        self.stack = []
        self.history = []
        self.n_j = n_j
        self.deck = []
        self.active_player = None
        self.last_player = None
        
        available_indices =list(range(40))
        
        for pinta in ['O','B','C','E']:
            for n in range(10):
                self.deck.append(str(n)+pinta)
    
    def deal(self):
        
        random.seed(seed)
        self.history = []
        self.players_data = []
        self.stack = []
        
        available_indices = list(range(40))
        
        for player in range(self.n_j):
            
            sample = random.sample(available_indices,int(40/self.n_j))
            self.players_data.append([self.deck[ind] for ind in sample])
            available_indices = list(set(available_indices) - set(sample))
            
    def active_player_0(self):
        
        if self.active_player == None:
            for player in range(self.n_j):
                if '0O' in self.players_data[player]:
                    self.active_player = player
        
    def possible_plays(self, player = False):
        
        if player == False:
            active_hand = self.players_data[self.active_player]
        else:
            active_hand = self.players_data[player]
            
        curr_quantity = len(self.stack)
        
        if curr_quantity == 0:
            if ('0O' in active_hand):
                pos_plays = [lst for lst in list(powerset([c for c in active_hand if c[0]=='0'])) if ('0O' in lst)]
                #for p in range(len(pos_plays)):
                    #print(str(p)+': ',pos_plays[p])
                return pos_plays          
            
        elif self.stack == ['*']:
            pos_plays = [lst for lst in list(powerset(active_hand)) if len(list(set([c[0] for c in lst])))==1]
            pos_plays.append([])
            return pos_plays
            
        else:
            curr_number = int(self.stack[0][0])
            pos_plays = [lst for lst in itertools.combinations(active_hand, curr_quantity) if (len(list(set([c[0] for c in lst])))==1) and (int(lst[0][0]) > curr_number)]
            
            #print([lst for lst in itertools.combinations(active_hand, curr_quantity) if (len(list(set([c[0] for c in lst])))==1)])
            pos_plays.append([])
            return pos_plays
     
    def play_select(self, selection = None):
        
        if self.last_player == self.active_player:
            self.stack = ['*']
            self.history.append('*')
            
        pos_plays = self.possible_plays()
        
        for ind in range(len(pos_plays)):
            print(ind,': ',pos_plays[ind])
        
        if selection == None:
            selection = int(input('Que jugada elige :'))
        
        if pos_plays[selection] != []:
        
            self.stack = list(pos_plays[selection])
            self.history.append(pos_plays[selection])
        
            self.last_player = self.active_player
            self.players_data[self.active_player] = list(set(self.players_data[self.active_player])-set(pos_plays[selection]))
        
        self.active_player = (self.active_player + 1)% self.n_j
        
    def hand_eval(self, player):
        
        hand_evaluation = 0
        
        for c in self.players_data[player]:
            
            hand_evaluation += -int(c[0])-1
            
        return hand_evaluation
        

In [133]:
game = culo_game(2)
game.deal(123)
game.active_player_0()

game.players_data[0].sort()
game.players_data[1].sort()

hand_0 = game.players_data[0] 
hand_1 = game.players_data[1]

print('Mano del jugador 0: \n',hand_0,'\n')
print(game.hand_eval(0),'\n')

print('Mano del jugador 0: \n',hand_1,'\n')


Mano del jugador 0: 
 ['0B', '0E', '1C', '1O', '2C', '2O', '3O', '4C', '4O', '5C', '5E', '5O', '6C', '6O', '7B', '7C', '7E', '7O', '8E', '9C'] 

-109 

Mano del jugador 0: 
 ['0C', '0O', '1B', '1E', '2B', '2E', '3B', '3C', '3E', '4B', '4E', '5B', '6B', '6E', '8B', '8C', '8O', '9B', '9E', '9O'] 



In [112]:
from gym import spaces

def culo_state_space(n_players):

    #
    # We define the matrix A which represents the sizes of the state_space
    # entries between 0 and 40*n_players represent a 0-1 encoding of player k's hand
    # entries between 40*n_players and 40*(n_players+1) represent the stack history
    # and the last 4 entries represent the stack number, the amount of cards in the
    # stack, the last player and the active player respectively.
    #

    A = 2*np.ones((40)*(n_players+1) + 4)
    A[40*(n_players)+1:40*(n_players+1)] = 10
    A[40*(n_players+1)] = 11
    A[40*(n_players+1)+1] = 5
    A[40*(n_players+1)+2] = n_players
    A[40*(n_players+1)+3] = n_players
    
    obs_space = spaces.MultiDiscrete(A)
    
    return obs_space


def culo_action_space(n_players):
    
    A = np.ones(3)
    
    A[0] = n_players
    A[1] = 5
    A[2] = 10
    
    action_space = spaces.MultiDiscrete(A)
    
    return(action_space)

def encode(game):
    
    state = np.zeros((40)*(game.n_j+1) + 4)
    
    for hand_index in range(len(game.players_data)):
        
        for card in game.players_data[hand_index]:
            
            state[(hand_index)*40 + game.deck.index(card)] = 1
    
    counter = 0
    
    for play in game.history:
        
        for card in play:
            
            state[40*(game.n_j)+counter] = int(card[0])
            
            counter += 1
    
    if game.stack == ['*'] or game.stack == []:
        state[40*(game.n_j+1)] = 10
    
    else:
        state[40*(game.n_j+1)] = int(game.stack[0][0])
    
    state[40*(game.n_j+1)] = len(game.stack)
    state[40*(game.n_j+1)+1] = game.last_player
    state[40*(game.n_j+1)+2] = game.active_player
    
    return state
    
# Get random samples from either space
# d = spaces.MultiDiscrete(state_space(2))
# print(d.sample())
# w = spaces.MultiDiscrete(action_space(2))
# print(w.sample())

In [113]:
import gym

In [114]:
class Culo(gym.Env):
    
    def __init__(self,n_j):
        
        self.n_j = n_j
        self.observation_space = culo_state_space(n_j)
        self.action_space = culo_action_space(n_j)
    
    def reset(self):
        
        self.game = culo_game(self.n_j)
        self.game.deal()
        self.game.active_player_0()
        
        self.state = encode(self.game)
        
    def step(self,action):
        
        player = action[0]
        amount_of_cards = action[1]
        card_number = action[2]
        
        pos_plays = self.game.possible_plays()
        
        if player == self.game.active_player:
            
            if amount_of_cards == 0:

                play = pos_plays.index([])
                self.game.play_select(play)
                reward = 0
                end = max([len(hand) for hand in self.game.players_data]) == 0
                self.state = encode(self.game)
                state = self.state
                
                return state, reward, end

            elif len([play for play in pos_plays if len([c for c in play if c[0]== card_number]) == amount_of_cards]) >= 1:

                play = pos_plays.index(random.choice([play for play in pos_plays if len([c for c in play if c[0]== card_number]) == amount_of_cards]))
                self.game.play_select(play)
                reward = amount_of_cards
                end = max([len(hand) for hand in self.game.players_data]) == 0
                self.state = encode(self.game)
                state = self.state
                
                return state, reward, end
        
            else:

                reward = -10
                self.state = encode(self.game)
                state = self.state
                
                return state, reward, False
        
        else:

            reward = -10
            self.state = encode(self.game)
            state = self.state
                
            return state, reward, False
        
        def render(self):
            
            print(f'El stack es {self.game.stack}\n')
            print(f'El último jugador fue {self.last_player}\n')
            print(f'El jugador activo es {self.active_player}\n')
            
            for player in range(self.n_j):
                self.game.players_data[player].sort()    
                print(f'La mano del jugador {player} es {self.game.players_data[player]}\n')
                print(f'Con evaluación {self.game.hand_eval(player)}\n')
            

In [115]:
from IPython.display import clear_output

env = Culo(2)

dim_obs = spaces.utils.flatdim(env.observation_space)
dim_action = spaces.utils.flatdim(env.action_space)

## definir q_table (hay que aplanar el vector del espacio)

q_table = np.zeros((env.observation_space.n, env.action_space.n))


# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

SyntaxError: invalid syntax (<ipython-input-115-246525a53c7f>, line 10)

In [116]:
from gym import spaces
d = spaces.MultiDiscrete([3,5,6])
spaces.utils.flatdim(d)

14

In [117]:
env = Culo(2)

dim_obs = env.observation_space.nvec

print(len(dim_obs))

dim_obs_nohistory = []

for x in range(124):
    if x<=79 or x>=120:
        dim_obs_nohistory.append(int(dim_obs[x]))

n_obs = math.prod(dim_obs_nohistory)

print(dim_obs_nohistory,n_obs)


dim_action = env.action_space.nvec

n_action = math.prod(dim_action)

print(dim_action,n_action)

q_table = sparse.csr_matrix((n_obs,n_action),dtype=int)

124
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 11, 5, 2, 2] 265963680315218418435358720
[ 2  5 10] 100


ValueError: Maximum allowed dimension exceeded

In [118]:
from scipy import sparse
M = sparse.csr_matrix((10, 10), dtype=int)

In [119]:
class culo_2(gym.Env):
    
    def __init__(self,hand):
        
        self.n_j = n_j
        self.observation_space = hand_space
        self.action_space = culo_action_space(n_j)
    
    def reset(self):
        
        self.game = culo_game(self.n_j)
        self.game.deal()
        self.game.active_player_0()
        
        self.state = encode(self.game)
        
    def step(self,action):
        
        player = action[0]
        amount_of_cards = action[1]
        card_number = action[2]
        
        pos_plays = self.game.possible_plays()
        
        if player == self.game.active_player:
            
            if amount_of_cards == 0:

                play = pos_plays.index([])
                self.game.play_select(play)
                reward = 0
                end = max([len(hand) for hand in self.game.players_data]) == 0
                self.state = encode(self.game)
                state = self.state
                
                return state, reward, end

            elif len([play for play in pos_plays if len([c for c in play if c[0]== card_number]) == amount_of_cards]) >= 1:

                play = pos_plays.index(random.choice([play for play in pos_plays if len([c for c in play if c[0]== card_number]) == amount_of_cards]))
                self.game.play_select(play)
                reward = amount_of_cards
                end = max([len(hand) for hand in self.game.players_data]) == 0
                self.state = encode(self.game)
                state = self.state
                
                return state, reward, end
        
            else:

                reward = -10
                self.state = encode(self.game)
                state = self.state
                
                return state, reward, False
        
        else:

            reward = -10
            self.state = encode(self.game)
            state = self.state
                
            return state, reward, False
        
        def render(self):
            
            print(f'El stack es {self.game.stack}\n')
            print(f'El último jugador fue {self.last_player}\n')
            print(f'El jugador activo es {self.active_player}\n')
            
            for player in range(self.n_j):
                self.game.players_data[player].sort()    
                print(f'La mano del jugador {player} es {self.game.players_data[player]}\n')
                print(f'Con evaluación {self.game.hand_eval(player)}\n')
            

In [156]:
hands_lst = []

for card_number in range(10):
    for card_amount_p1 in range(5):
        for card_amount_p2 in range(5):
            for card_amount_history in range(5):
                if card_amount_p1+card_amount_p2+card_amount_history== 4:
                    hands_lst.append((card_number,card_amount_p1,card_amount_p2,card_amount_history))


stack_lst = []
for curr_player in range(2):
    for starts in range(2):
        for card_number in range(10):
            for card_amount in range(1,5):
                    stack_lst.append([(card_number,card_amount),(curr_player,starts)])
                        
#print(hands_lst,stack_lst)

def translate_state(state):
    
    hand_data = hands_lst[state]
    
    return hand_data

def translate_stack(stack):
        
    stack_data = stack_lst[stack]
    
    return stack_data

def encode_data(hand_data,stack_data,history):

    vec_hand = 0 #definir
    
    indices = np.zeros(2,dtype='int16')
    
    indices[0] = hands_lst.index(vec_hand)
    
    indices[1] = stack_lst.index(vec_stack)
        
    return indices
            
print(f'Para 2 jugadores hand_lst tiene largo {len(hands_lst)} y stack_lst tiene largo {len(stack_lst)}, el espacio de estados del juego es un elemento de tamaño {len(hands_lst)*len(stack_lst)}. Un estado es una tupla (s,a), donde 0<=s<={len(hands_lst)} y 0<=a<={len(stack_lst)}, la funcón translate_state entrega para un estado s, la coleccion de manos de los jugadores en el estado s. \n ')
print(f'Por ejemplo para el estado s={50}, la función translate_state(50) entrega {translate_state(50)}, que representa que el número #3 es tal que el jugador 1 tiene 1 en la mano, el jugador 2 tiene 0 en la mano, y se han jugado 3 en la historia del juego, si por otro lado a={50}, tenemos que translate_stack(50) es {translate_stack(50)}, este estado significa que en la pila hay #3 copias del número #0, le toca al jugador 1 y parte la ronda.\n')


print(f'Simularemos un juego:\n')

seed=999999
game = culo_game(2)
game.deal(seed)
game.active_player_0()

game.players_data[0].sort()
game.players_data[1].sort()

hand_0 = game.players_data[0] 
hand_1 = game.players_data[1]

game_history = game.history

print('La historia del juego es: ', game_history,'\n')

print('Mano inicial del jugador 0: \n',hand_0,'\n')

print('Mano inicial del jugador 1: \n',hand_1,'\n')

amount=[]

for n in range(10):
    amount.append((n,len([c for c in hand_0 if int(c[0])==n]),len([c for c in hand_1 if int(c[0])==n]),0))

print(f'Codificada en el vector amount = {amount}')

Para 2 jugadores hand_lst tiene largo 150 y stack_lst tiene largo 160, el espacio de estados del juego es un elemento de tamaño 24000. Un estado es una tupla (s,a), donde 0<=s<=150 y 0<=a<=160, la funcón translate_state entrega para un estado s, la coleccion de manos de los jugadores en el estado s. 
 
Por ejemplo para el estado s=50, la función translate_state(50) entrega (3, 1, 0, 3), que representa que el número #3 es tal que el jugador 1 tiene 1 en la mano, el jugador 2 tiene 0 en la mano, y se han jugado 3 en la historia del juego, si por otro lado a=50, tenemos que translate_stack(50) es [(2, 3), (0, 1)], este estado significa que en la pila hay #3 copias del número #0, le toca al jugador 1 y parte la ronda.

Simularemos un juego:

La historia del juego es:  [] 

Mano inicial del jugador 0: 
 ['0B', '0O', '1B', '3B', '3E', '4B', '4C', '4O', '5B', '5E', '5O', '6B', '6C', '6E', '6O', '7B', '7C', '8C', '9B', '9E'] 

Mano inicial del jugador 1: 
 ['0C', '0E', '1C', '1E', '1O', '2B', 

In [154]:




def init_q_table():
    
    A = np.random.uniform(size=(len(hands_lst),len(stack_lst)))
    
    return A



#print(len(hands_lst),hands_lst)
#print(len(stack_lst),stack_lst)